In [1]:
from feast import FeatureStore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from joblib import dump
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from joblib import load

In [2]:
# Getting our FeatureStore
store = FeatureStore(repo_path=".")

In [3]:
# Retrieving the saved dataset and converting it to a DataFrame
training_df = store.get_saved_dataset(name="churn_dataset").to_df()

C:\Users\claud\AppData\Local\Programs\Python\Python311\Lib\site-packages\feast\feature_store.py:1178: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [4]:
training_df

,event_timestamp,Gender,Exited,IsActiveMember,Geography_France,Age,CreditScore,Tenure,Balance,Geography_Germany,EstimatedSalary,USER_ID,NumOfProducts,HasCrCard,Geography_Spain
0,2023-01-01 00:00:00+00:00,1,0,0,False,37,648,6,131753.41,True,86894.67,9417,1,1,False
1,2023-01-01 00:00:00+00:00,1,0,1,False,36,699,6,147137.74,True,33687.90,6186,1,1,False
2,2023-01-01 00:00:00+00:00,1,0,1,True,49,611,3,0.00,False,142917.54,792,2,1,False
3,2023-01-01 00:00:00+00:00,0,0,0,True,33,635,5,0.00,False,122949.71,9133,2,1,False
4,2023-01-01 00:00:00+00:00,1,0,1,False,43,818,10,105301.50,True,78941.59,9145,1,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2023-12-31 00:00:00+00:00,0,0,0,False,35,540,7,128369.75,True,198256.15,9329,2,1,False
9996,2023-12-31 00:00:00+00:00,1,0,0,False,24,742,8,0.00,False,4070.28,7324,2,1,True
9997,2023-12-31 00:00:00+00:00,0,1,0,True,51,625,4,124620.01,False,92243.94,7966,2,1,False
9998,2023-12-31 00:00:00+00:00,1,0,0,False,20,463,8,204223.03,False,128268.39,5255,1,1,True


In [5]:
# Separating the features and labels
labels = training_df['Exited']
features = training_df.drop(
    labels=['Exited', 'event_timestamp', 'USER_ID'],
    axis=1)

In [6]:
# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    stratify=labels)

In [7]:
columns_order = load('columns_order.joblib')
columns_order

Index(['Geography_Spain', 'HasCrCard', 'Balance', 'Age', 'Gender', 'Tenure',
       'EstimatedSalary', 'CreditScore', 'NumOfProducts', 'Geography_France',
       'IsActiveMember', 'Geography_Germany'],
      dtype='object')

In [8]:
# Riorganizza le colonne di X_test in base all'ordine delle colonne utilizzate durante l'addestramento
X_test = X_test[columns_order]
X_train = X_train[columns_order]

### Train - Logistic Regression Classifier

In [9]:
# Define the hyperparameter grid
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 300]
}

In [10]:
# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search to your data
grid_search.fit(X=X_train, y=y_train)

C:\Users\claud\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\claud\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\claud\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\claud\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\claud\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the 

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1, 10, 100], 'max_iter': [100, 200, 300],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [11]:
# Supponiamo che queste siano le colonne usate durante l'addestramento
columns_order = X_train.columns

# Salva l'ordine delle colonne in un file di testo
with open("columns_order2.txt", "w") as file:
    for column in columns_order:
        file.write(f"{column}\n")

In [12]:
columns_order

Index(['Geography_Spain', 'HasCrCard', 'Balance', 'Age', 'Gender', 'Tenure',
       'EstimatedSalary', 'CreditScore', 'NumOfProducts', 'Geography_France',
       'IsActiveMember', 'Geography_Germany'],
      dtype='object')

In [13]:
# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print("Best Parameters: ", best_params)
print("Best Estimator: ", best_estimator)

Best Parameters:  {'C': 100, 'max_iter': 300, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Estimator:  LogisticRegression(C=100, max_iter=300)


In [14]:
# Make predictions and evaluate your model
y_pred = best_estimator.predict(X_test)

In [15]:
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: ", accuracy)

Test Accuracy:  0.812


In [16]:
# Saving the model
dump(value=grid_search, filename="logistic_regression.joblib")

['logistic_regression.joblib']

In [17]:
dump(X_train.columns, 'columns_order.joblib')

['columns_order.joblib']